In [13]:
import csv
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from skimage.feature import canny
from skimage.filters import sobel
from skimage.transform import hough_line, hough_line_peaks
from skimage.draw import polygon
from matplotlib import pylab as pylab

miss_table_path = "..\\..\\datasets\\table\\miss_metadata.csv"
clean_table_path = "..\\..\\datasets\\table\\clean_metadata.csv"
miss_png_path = "..\\..\\datasets\\image\\miss_png\\"
clean_png_path = "..\\..\\datasets\\image\\clean_png\\"
miss_bilateral_png = "..\\..\\datasets\\image\\miss_bilateral_png\\"
clean_bilateral_png = "..\\..\\datasets\\image\\clean_bilateral_png\\"

miss_ori = list(map(lambda x: x.split('/')[9], pd.read_csv(miss_table_path, dtype=str)['anon_dicom_path']))
miss_ori_pos = list(map(lambda x: x, pd.read_csv(miss_table_path, dtype=str)['ViewPosition']))
miss = zip(miss_ori, miss_ori_pos)
clean_ori = list(map(lambda x: x.split('/')[9], pd.read_csv(clean_table_path, dtype=str)['anon_dicom_path']))
clean_ori_pos = list(map(lambda x: x, pd.read_csv(clean_table_path, dtype=str)['ViewPosition']))
clean = zip(clean_ori, clean_ori_pos)

def bilateral_filter(ori_path):  
    ori_img = cv2.imread(ori_path, 0)
    # print(ori_path,'to', bil_path)
    bil_img = cv2.bilateralFilter(ori_img, 9, 150, 150)
    return bil_img

def select_breast_area(bil_img):
    # binarization: use thresholding to create a binary mask
    th, img_binary = cv2.threshold(bil_img, 7, 255, cv2.THRESH_BINARY)
    # expand the border of white contours (dliate -> open)
    kernel = np.ones((21,21),np.uint8)
    img_opening = cv2.morphologyEx(img_binary, cv2.MORPH_OPEN, kernel)
    # deciding the breast area with binary mask
    img_masked = cv2.bitwise_and(bil_img,img_opening)
    return img_masked

def clahe(img):
    equ = cv2.equalizeHist(img)
    clahe = cv2.createCLAHE(clipLimit =3.0, tileGridSize=(4,4))
    cl_img = clahe.apply(img)
    ret, thresh3 = cv2.threshold(cl_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return cl_img

def apply_canny(image):
    # img = cv2.medianBlur(image, 9)                 # 模糊化，去除雜訊
    output = cv2.Canny(image, 10, 35) 
    return output

def get_hough_lines(canny_img):
    h, theta, d = hough_line(canny_img)
    lines = list()
    print('\nAll hough lines')
    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        print("Angle: {:.2f}, Dist: {:.2f}".format(np.degrees(angle), dist))
        x1 = 0
        y1 = (dist - x1 * np.cos(angle)) / np.sin(angle)
        x2 = canny_img.shape[1]
        y2 = (dist - x2 * np.cos(angle)) / np.sin(angle)
        lines.append({
            'dist': dist,
            'angle': np.degrees(angle),
            'point1': [x1, y1],
            'point2': [x2, y2]
        })
    
    return lines

def shortlist_lines(lines):
    MIN_ANGLE = 15
    MAX_ANGLE = 90
    MIN_DIST  = 750
    MAX_DIST  = 1000
    
    shortlisted_lines = [x for x in lines if 
                          (x['dist']>=MIN_DIST) &
                          (x['dist']<=MAX_DIST) &
                          (x['angle']>=MIN_ANGLE) &
                          (x['angle']<=MAX_ANGLE)
                        ]
    print('\nShorlisted lines')
    for i in shortlisted_lines:
        print("Angle: {:.2f}, Dist: {:.2f}".format(i['angle'], i['dist']))
        
    return shortlisted_lines

def remove_pectoral(shortlisted_lines):
    shortlisted_lines.sort(key = lambda x: x['dist'])
    # print(shortlisted_lines)
    pectoral_line = shortlisted_lines[0]
    d = pectoral_line['dist']
    theta = np.radians(pectoral_line['angle'])
    
    x_intercept = d/np.cos(theta)
    y_intercept = d/np.sin(theta)
    
    return polygon([0, 0, y_intercept], [0, x_intercept, 0])

def hough(image, cla_image):
    rows, cols = image.shape

    # Step 2: Contour detection using Canny filter
    img = cv2.medianBlur(image, 33)  
    canny_image = apply_canny(img)

    # Step 3: Linear aperture filtering
    filtered_image = cv2.filter2D(canny_image, -1, np.array([[0, 2, 0], [2, 2, 2], [0, 2, 0]]))

    # Step 4: Line detection using Hough Transform
    lines = get_hough_lines(filtered_image)
    shortlisted_lines = shortlist_lines(lines)
    
    # Step 5: Apply the Hough mask to the region of interest
    rr, cc = remove_pectoral(shortlisted_lines)
    # Create a boolean mask for the conditions
    mask = (rr < rows) & (cc < cols)
    # Set values to 0 where the conditions are met
    cla_image[rr[mask], cc[mask]] = 0
    return cla_image

# test_pic = "../../datasets/image/clean_png/1.2.826.0.1.3680043.8.498.21419879400540352363681436133028226505.dcm.png"
# new_path = "../../datasets/image/clean_png/test5_clahe.png"

# bil_img = bilateral_filter(test_pic)
# masked_img = select_breast_area(bil_img)
# cl_img = clahe(masked_img)
# cv2.imwrite(new_path, cl_img)

for name, pos in clean:
    ori_path = clean_png_path+name+".png"
    new_path = clean_bilateral_png+name+".png"
    bil_img = bilateral_filter(ori_path)
    masked_img = select_breast_area(bil_img)
    clahe_img = clahe(masked_img)
    if pos == 'MLO':
        rows, cols = clahe_img.shape
        roi = clahe_img[:2 * rows//3, :cols//3]
        hough_img = hough(roi, clahe_img)
        cv2.imwrite(new_path, hough_img)
    else:
        cv2.imwrite(new_path, clahe_img)

for name, pos in miss:
    ori_path = miss_png_path+name+".png"
    new_path = miss_bilateral_png+name+".png"
    bil_img = bilateral_filter(ori_path)
    masked_img = select_breast_area(bil_img)
    clahe_img = clahe(masked_img)
    if pos == 'MLO':
        rows, cols = clahe_img.shape
        roi = clahe_img[:2 * rows//3, :cols//3]
        hough_img = hough(roi, clahe_img)
        cv2.imwrite(new_path, hough_img)
    else:
        cv2.imwrite(new_path, clahe_img)


True